# 3.4 Relationships Between Quantitative and Categorical Variables

In this section, we discuss ways to visualize the relationship between a categorical variable, like the building type of a home, and a quanitative variable, like the sale price.

In [ ]:
import pandas as pd

data_dir = "https://dlsun.github.io/pods/data/"
df_ames = pd.read_csv(data_dir + "AmesHousing.txt", sep="\t")
df_ames.head()

One way is to visualize the conditional distributions of the quantitative variable, given the value of the categorical variable. In order to do this, we have to first determine the possible values of the categorical variable.

In [ ]:
df_ames["Bldg Type"].unique()

Now, we can make a histogram of the sale price for each building type, by repeatedly subsetting the data set.

In [ ]:
for bldg_type in df_ames["Bldg Type"].unique():
    df_ames[df_ames["Bldg Type"] == bldg_type]["SalePrice"].plot.hist(
        label=bldg_type,
        density=True,
        alpha=0.5,
        legend=True
    )

A density plot is less busy and may make it easier to compare the distributions of sale price for different building types.

In [ ]:
for bldg_type in df_ames["Bldg Type"].unique():
    df_ames[df_ames["Bldg Type"] == bldg_type]["SalePrice"].plot.density(
        label=bldg_type,
        legend=True,
        xlim=(0, 600000)
    )

If we only need to know the center of each distribution, we can simply report the conditional mean for each building type. The conditional means can be visualized using a bar chart. We manually construct the `pandas` `Series` for making the bar chart.

In [ ]:
mean_sale_price = pd.Series()

for bldg_type in df_ames["Bldg Type"].unique():
    mean_sale_price[bldg_type] = df_ames[df_ames["Bldg Type"] == bldg_type]["SalePrice"].mean()
    
mean_sale_price.plot.bar()
mean_sale_price

However, this code is inefficient and repetitive. In this lesson, we will learn a framework for concisely specifying analyses like the one above.

## Split-Apply-Combine Strategy

The problem of calculating the survival rate by class is an example of a problem that can be solved using the **split-apply-combine strategy**. The key insight here is that many data analyses follow the same basic pattern:

- First, a data set is **split** into several subsets based on some variable.
- Next, some analysis is **applied** to each subset.
- Finally, the results from each analysis are **combined**.

The three steps are diagrammed in the figure below:

![](split_apply_combine.png) [source](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb)

Applying this strategy to our working example above, we should first _split_ up the Titanic data according to the value of `pclass`, _apply_ `.survived.mean()` to each subset, and finally _combine_ the results into one `Series`.

[_Note:_ The term "split-apply-combine" was coined by Hadley Wickham in [a 2011 paper](https://www.jstatsoft.org/article/view/v040i01), but the idea is not new. It should already be familiar to you if you know SQL or MapReduce.]

## Split-Apply-Combine in `pandas`: the `.groupby()` method

To implement the split-apply-combine strategy in `pandas`, we use the `.groupby()` method. First, we specify one or more variables to split on in the argument to `.groupby()`. Then, we specify our analysis as usual. Pandas will handle splitting the data, applying the analysis to each subset, and combining the results at the end.

For example, to make the histograms of sale price for each building type, we can `.groupby` the building type and plot as usual. Note that we do not need to specify the labels; `pandas` will automatically use the different building types as the labels.

In [ ]:
df_ames.groupby("Bldg Type")["SalePrice"].plot.hist(
        density=True,
        alpha=0.5,
        legend=True
    )

Compare this line of code with the code to plot a histogram of **SalePrice**:

`df_ames["SalePrice"].plot.hist(...)`.

The only difference is `.groupby("Bldg Type")`. This turns a `DataFrame` into a `DataFrameGroupBy` object, which behaves like a `DataFrame`, except that any analysis that we specify will be applied to every subset of the `DataFrame` instead of the whole `DataFrame`.

Similarly, to calculate the average sale price for each building type, we can `.groupby` the building type and calculate the mean as usual.

In [ ]:
df_ames.groupby("Bldg Type")["SalePrice"].mean()

This automatically returned the desired `Series` that we can pass to `.plot.bar()`.

In [ ]:
(df_ames.groupby("Bldg Type")["SalePrice"].mean().
 plot.bar())

It is also possible to group by more than one variable. Simply pass in a list of variable names to `.groupby()`. For example, the following code calculates the average sale price by building type and house style.

In [ ]:
df_ames.groupby(["Bldg Type", "House Style"])["SalePrice"].mean()

Notice that when we use `.groupby()`, the resulting index is whatever variable(s) we grouped by. Since we grouped by two variables, this index actually has two levels. An index with more than one level is called a `MultiIndex` in `pandas`. To access a particular row in a `DataFrame` that is indexed by a `MultiIndex`, we pass in a tuple of the values we want from each level.

In [ ]:
df_ames.groupby(["Bldg Type", "House Style"])["SalePrice"].mean()[
    ("1Fam", "2Story")
]

# Exercises

Exercises 1-5 work with the Tips data set (`https://dlsun.github.io/pods/data/tips.csv`).

1\. On which day of the week does the waiter serve the largest parties, on average?

2\. Calculate the average bill by day and time. What day-time combination has the highest average bill?

3\. Extract the average bill for Friday lunch from the result of Exercise 2.

4\. Make a visualization comparing the distribution of tip percentages left by males and females. How do they compare?

5\. Make a visualization that shows the average tip percentage as a function of table size.